1. Number of updated projects of both sheets since and before (by donor agency and by SWG)(The number of updated projects refers to both the online Excel sheet and Google form)
2. Number of updated projects on the online Excel sheet only (whose online form is not updated)( by donor agency and by SWG)
3. Number of new projects and list registered ( by donor agency and by SWG)
4. Number of not updated projects ( by donor agency and by SWG)
5. Can we get an Excel sheet under each SWG updated and not updated project info of both online and Excel sheet, updated fields colored?
6. Can we get all SWG info in one file in the above similar request?

In [1]:
import pandas as pd
import re
from openpyxl.styles import PatternFill, Border, Side, Alignment, Font
from tqdm import tqdm
import os
from glob import glob


def format_excel(writer, sheet_name):
    worksheet = writer.sheets[sheet_name]
    header_fill = PatternFill(start_color="D7E4BC", end_color="D7E4BC", fill_type="solid")
    thin_border = Border(left=Side(style='thin'), 
                        right=Side(style='thin'), 
                        top=Side(style='thin'), 
                        bottom=Side(style='thin'))
    header_font = Font(bold=True)
    header_alignment = Alignment(horizontal='center', vertical='top', wrap_text=True)

    for cell in worksheet[1]:
        cell.fill = header_fill
        cell.border = thin_border
        cell.font = header_font
        cell.alignment = header_alignment

    worksheet.auto_filter.ref = "A1:S1"
    worksheet.freeze_panes = 'A2' 

    high = ['A', 'B']  
    medium = ['F', 'G', 'H', 'L', 'M', 'K',  'I', 'J', 'Q', 'R', 'S', 'T'] 
    low = ['C', 'D', 'E', 'N', 'O', 'P']

    for col in high:
        worksheet.column_dimensions[col].width = 30
    for col in medium:
        worksheet.column_dimensions[col].width = 18 
    for col in low:
        worksheet.column_dimensions[col].width = 12    

def get_ids_and_titles(original, latest):
    original_titles = original.iloc[7:, 2].values
    latest_titles = latest.iloc[7:, 2].values
    original_ids = original.iloc[7:, 7].values
    latest_ids = latest.iloc[7:, 7].values
    return original_titles, latest_titles, original_ids, latest_ids   

def compare_stat(original, latest):
    new_projs = {}
    original_not_updated_projs = {}
    original_updated_projs = {}
    deleted_projs = {}

    _, latest_titles, original_ids, latest_ids = get_ids_and_titles(original, latest)

    for i, original_id in enumerate(original_ids):
        if original_id not in latest_ids:
            deleted_projs[original_id] = original.iloc[i + 7]

    for i, latest_id in enumerate(latest_ids): 
        if latest_id in original_ids:
            original_index = original[original.iloc[:, 7] == latest_id].index[0]
            original_row = original.iloc[original_index]
            latest_index = latest[latest.iloc[:, 7] == latest_id].index[0]
            latest_row = latest.iloc[latest_index]
            if original_row.equals(latest_row):
                original_not_updated_projs[latest_id] = original_row
            else:
                original_updated_projs[latest_id] = latest_row

        elif pd.isna(latest_id) or latest_id.isspace() or bool(re.search(r'[^a-zA-Z0-9\-]', latest_id)) or latest_id == '':
            if latest_titles[i] not in new_projs: 
                new_projs[latest_titles[i]] = latest.iloc[i + 7]
            new_projs[latest_titles[i]] = latest.iloc[i + 7]
        else:
            print(f'###### ID ERROR. Id: {latest_id} Title: {latest_titles[i]} ######')                       

    return new_projs, deleted_projs, original_not_updated_projs, original_updated_projs

def process_files(orig_file, lat_file, output_fname):
    original_sinc = pd.read_excel(orig_file, sheet_name=sincesht)
    latest_sinc = pd.read_excel(lat_file, sheet_name=sincesht)
    original_befo = pd.read_excel(orig_file, sheet_name=beforesht)
    latest_befo = pd.read_excel(lat_file, sheet_name=beforesht)
    original_new = pd.read_excel(orig_file, sheet_name=newsht)
    latest_new = pd.read_excel(lat_file, sheet_name=newsht)

    new_project_sinc, deleted_project_sinc, original_not_updated_projs_sinc, original_updated_projs_sinc = compare_stat(original_sinc, latest_sinc)
    new_project_befo, deleted_project_befo, original_not_updated_projs_befo, original_updated_projs_befo = compare_stat(original_befo, latest_befo)
    new_project_new, deleted_project_new, original_not_updated_projs_new, original_updated_projs_new = compare_stat(original_new, latest_new)

    output_dir = os.path.dirname(output_fname)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    try:
        with pd.ExcelWriter(output_fname) as writer:
            pd.DataFrame.from_dict(new_project_sinc, orient='index').to_excel(writer, sheet_name='New Projs Since 2017')
            pd.DataFrame.from_dict(deleted_project_sinc, orient='index').to_excel(writer, sheet_name='Deleted Projs Since 2017')
            pd.DataFrame.from_dict(original_updated_projs_sinc, orient='index').to_excel(writer, sheet_name='Updated Projs Since 2017')
            pd.DataFrame.from_dict(original_not_updated_projs_sinc, orient='index').to_excel(writer, sheet_name='Not Updated Projs Since 2017')

            pd.DataFrame.from_dict(new_project_befo, orient='index').to_excel(writer, sheet_name='New Projs Before 2017')
            pd.DataFrame.from_dict(deleted_project_befo, orient='index').to_excel(writer, sheet_name='Deleted Projs Before 2017')
            pd.DataFrame.from_dict(original_updated_projs_befo, orient='index').to_excel(writer, sheet_name='Updated Projs Before 2017')
            pd.DataFrame.from_dict(original_not_updated_projs_befo, orient='index').to_excel(writer, sheet_name='Not Updated Projs Before 2017')

            pd.DataFrame.from_dict(new_project_new, orient='index').to_excel(writer, sheet_name='New Projs new')

        df = pd.read_excel(output_fname, sheet_name=None)

        sheets_to_exclude = []
        for sheet in df:
            if df[sheet].shape[1] == 0:
                sheets_to_exclude.append(sheet)
                continue
            df[sheet] = df[sheet].iloc[:, 2:-1]
            if df[sheet].shape[1] > len(cols):
                df[sheet] = df[sheet].iloc[:, :len(cols)]
            elif df[sheet].shape[1] < len(cols):
                for col in cols[df[sheet].shape[1]:]:
                    df[sheet][col] = pd.NA
            df[sheet].columns = cols

        sheets_to_merge = ['New Projs Since 2017', 'New Projs Before 2017', 'New Projs new']
        merged_df = pd.concat([df[s] for s in sheets_to_merge if s not in sheets_to_exclude], axis=0)
        df['New Projects'] = merged_df[cols]
        sheets_to_exclude.extend(sheets_to_merge)

        with pd.ExcelWriter(output_fname) as writer:
            for sheet in df:
                if sheet in sheets_to_exclude:
                    # print(f'###### Empty Sheet {sheet} has been excluded ######')
                    continue
                df[sheet].to_excel(writer, sheet_name=sheet, index=False)
                format_excel(writer, sheet)    

    except Exception as e:
        print("Error occurred while writing to Excel:", e)        

cols = ['SWG', 'Project Title', 'Status', 'On/Off/Treasury Budget', 'Humanitarian Aid', 'Project description', 
        'Project ID (in AMP)', 'Actual Approval Date', 'Actual Start Date', 'Actual Closure Date', 
        'Donor Group', 'Donor Agency', 'Implementing Agency', 'Type of Assistance', 'Mode of Payment', 
        'Aid Modality', 'Actual Commitments', 'Actual Disbursements', 'Undisbursed Balance']

sincesht = 'Existing projects approved sinc'
beforesht = 'Existing projects approved befo'
newsht = 'Record new projects'

file_pairs = [
    ('ODA_Original_and_Latest/OrigHPN.xlsx', 'ODA_Original_and_Latest/LatHPN.xlsx', 'Report/HPN.xlsx'),
    ('ODA_Original_and_Latest/OrigBSD.xlsx', 'ODA_Original_and_Latest/LatBSD.xlsx', 'Report/BSD.xlsx'),
    ('ODA_Original_and_Latest/OrigREDFS.xlsx', 'ODA_Original_and_Latest/LatREDFS.xlsx', 'Report/REDFS.xlsx'),
    ## Add more file pairs here
]

tqdm.write('Processing files...')
for orig_file, lat_file, output_fname in tqdm(file_pairs):
    process_files(orig_file, lat_file, output_fname)
tqdm.write('Processing Done!')    


directory = 'Report'
excel_files = glob(os.path.join(directory, '*.xlsx'))
merged_data = {}

for file in excel_files:
    xls = pd.ExcelFile(file)
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(file, sheet_name=sheet_name)
        if sheet_name in merged_data:
            merged_data[sheet_name] = pd.concat([merged_data[sheet_name], df])
        else:
            merged_data[sheet_name] = df

with pd.ExcelWriter('Report/All.xlsx') as writer:
    for sheet_name, data in merged_data.items():
        data.to_excel(writer, sheet_name=sheet_name, index=False)

with pd.ExcelWriter('Report/All.xlsx') as writer:
    for sheet in merged_data:
        merged_data[sheet].to_excel(writer, sheet_name=sheet, index=False)
        format_excel(writer, sheet)
    
merged_file_path = 'Report/All.xlsx'
response_file_path = 'ODA_Original_and_Latest/Response.xlsx'
output_file_path = 'Report/All_with_Response.xlsx'

response_sheet = pd.read_excel(response_file_path, sheet_name='Form Responses 1')
response_values = response_sheet[response_sheet.columns[3]].astype(str).str.strip()

merged_sheets = pd.read_excel(merged_file_path, sheet_name=None)

tobe_formated = None
with pd.ExcelWriter(output_file_path) as writer:
    for sheet_name, merged_data in merged_sheets.items():
        merged_values = merged_data[merged_data.columns[1]].astype(str).str.strip()
        output_rows = []

        for merged_index, merged_val in merged_values.items():
            for response_index, response_val in response_values.items():
                if response_val.endswith('...') and response_val[:-3] in merged_val:
                    combined_row = merged_data.loc[merged_index].to_dict()
                    combined_row.update(response_sheet.loc[response_index].to_dict())
                    combined_row[response_sheet.columns[3]] = merged_val
                    output_rows.append(combined_row)
                elif response_val == merged_val or response_val in merged_val or merged_val in response_val:
                    combined_row = merged_data.loc[merged_index].to_dict()
                    combined_row.update(response_sheet.loc[response_index].to_dict())
                    output_rows.append(combined_row)

        if output_rows:
            output_df = pd.DataFrame(output_rows)
            print(f'Found {output_df.shape[0]} matching rows in {sheet_name}')
            output_df.to_excel(writer, sheet_name=sheet_name, index=False)

print('Done!')


Processing files...


  0%|          | 0/3 [00:00<?, ?it/s]

###### Empty Sheet New Projs Since 2017 has been excluded ######
###### Empty Sheet New Projs Before 2017 has been excluded ######
###### Empty Sheet Deleted Projs Before 2017 has been excluded ######
###### Empty Sheet New Projs new has been excluded ######


 67%|██████▋   | 2/3 [00:15<00:07,  7.21s/it]

###### Empty Sheet New Projs Since 2017 has been excluded ######
###### Empty Sheet Deleted Projs Since 2017 has been excluded ######
###### Empty Sheet New Projs Before 2017 has been excluded ######
###### Empty Sheet Deleted Projs Before 2017 has been excluded ######
###### Empty Sheet Updated Projs Before 2017 has been excluded ######
###### Empty Sheet New Projs new has been excluded ######
###### Empty Sheet New Projs Since 2017 has been excluded ######
###### Empty Sheet Deleted Projs Since 2017 has been excluded ######
###### Empty Sheet New Projs Before 2017 has been excluded ######
###### Empty Sheet Deleted Projs Before 2017 has been excluded ######
###### Empty Sheet Updated Projs Before 2017 has been excluded ######
###### Empty Sheet New Projs new has been excluded ######


100%|██████████| 3/3 [00:24<00:00,  8.24s/it]


Done!
Found 61 matching rows in Updated Projs Since 2017
Found 18 matching rows in Not Updated Projs Since 2017
Found 118 matching rows in New Projects
Found 27 matching rows in Updated Projs Before 2017
Done!
